In [1]:
import argparse
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
from intern.utils.parallel import block_compute
import configparser
import requests
import numpy as np
from numpy import genfromtxt
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime
import seaborn as sns
from intern.utils.parallel import block_compute
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
CONFIG_FILE = 'config.ini'

In [2]:
COLL_NAME      = 'collman' 
EXP_NAME       = 'M247514_Rorb_1_Site3Align2_EM'
ANNO_NAME      = 'm247514_Site3Annotation_MN_global'
COORD_FRAME    = 'collman_M247514_Rorb_1_Site3Align2_EM_m247514_Site3Annotation_MN_global'
CONFIG_FILE    = 'config.ini'
OUTPUT         = 'testCollman14v2Cubes.csv'

#CHAN_NAMES = ['Synapsin647']
#CHAN_NAMES = ['DAPI1st', 'DAPI2nd', 'DAPI3rd', 'GABA488', 'GAD647',
#        'gephyrin594', 'GS594', 'MBP488', 'NR1594', 'PSD95_488',
#        'Synapsin647', 'VGluT1_647']
#CHAN_NAMES = ['synapsin', 'PSDr'] 

In [26]:
bf = [5,108,108] # in z,y,x order
                                                                                                     
config = configparser.ConfigParser()
config.read(CONFIG_FILE)
TOKEN = config['Default']['token']
boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
#print(boss_url)
#'https://api.boss.neurodata.io/v1/'

#intern
rem = BossRemote(CONFIG_FILE)
                                                                                                     
cf = CoordinateFrameResource(str(COLL_NAME + '_' + EXP_NAME))
cfr = rem.get_project(cf)
anno_res = ChannelResource(ANNO_NAME, COLL_NAME, EXP_NAME, 'annotation', datatype='uint64')

ex = {'x': cfr.x_stop, 'y': cfr.y_stop, 'z': cfr.z_stop}

blocks = block_compute(0,ex['x'],0,ex['y'],0,ex['z'],
           origin = (0,0,0), block_size = (512, 512, 16))
                                                                                                     
rid = []
for b in blocks:
    tmp = rem.get_ids_in_region(anno_res, 0, b[0], b[1], b[2], [0,1])
    if tmp != []:
        rid = rid + tmp
                                                                                                     
u = np.unique(np.asarray(rid))
u_old = u
                                                                                                     
## bounding box for annotation_i
bb = [rem.get_bounding_box(anno_res, 0,ui, 'tight') for ui in u]
                                                                                                     
for i in range(len(bb)):
    bb[i]["id"] = u[i]

KeyboardInterrupt: 

In [27]:
blocks[0]
b0 = ((0, ))

((512, 1024), (512, 1024), (16, 32))

In [15]:
A = [(rem.get_cutout(
      anno_res, 0, bb[i]["x_range"], 
      bb[i]["y_range"], bb[i]["z_range"], 
      id_list = [bb[i]['id']]) == bb[i]['id']).astype(int) 
    for i in range(len(bb))] 
                                                                                                     
#Bmeans = [np.int32(np.round(np.mean(np.asarray(np.where(A[i] == True)),1))) for i in range(len(A))]
Bmeans = [np.uint32(np.round(np.mean(np.asarray(np.where(A[i] == 1)),1))) for i in range(len(A))]

Bglobal = []
for i in range(len(bb)):
    ad1 = np.asarray([bb[i]['z_range'][0], bb[i]['y_range'][0], bb[i]['x_range'][0]])
    Bglobal.append(Bmeans[i] + ad1)
    
ColMin = np.asarray(bf)
ColMax = np.asarray([ex['z'] - (bf[0] + 1),  # The z index is inclusive
                     ex['y'] - (bf[1] + 1), 
                     ex['x'] - (bf[2] + 1)])
                                                                                                     
m = [Bglobal[i] >= ColMin for i in range(len(Bglobal))]
M = [Bglobal[i] <= ColMax for i in range(len(Bglobal))]
mm = [np.all(m[i]) for i in range(len(m)) ]
MM = [np.all(M[i]) for i in range(len(M)) ]
                                                                                                     
                                                                                                     

In [18]:
A[0].shape

(2, 65, 49)

In [20]:
Bglobal[0]

array([  12, 1026,  882])